
## Rasa NLU & Rasa Core - Building Chatbots



In [1]:
# Installation
!pip install rasa_core==0.9.6 rasa_nlu==0.14.4 sklearn_crfsuite

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

    100% |████████████████████████████████| 133kB 5.9MB/s 
    100% |████████████████████████████████| 153kB 13.0MB/s 
    100% |████████████████████████████████| 49.1MB 463kB/s 
    100% |████████████████████████████████| 51kB 20.1MB/s 
    100% |████████████████████████████████| 71kB 25.6MB/s 
    100% |████████████████████████████████| 962kB 13.2MB/s 
    100% |████████████████████████████████| 71kB 24.3MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
    100% |████████████████████████████████| 655kB 17.5MB/s 
    100% |████████████████████████████████| 307kB 22.2MB/s 
    100% |████████████████████████████████| 4.9MB 5.6MB/s 
    100% |████████████████████████████████| 12.8MB 2.2MB/s 
    100% |████████████████████████████████| 81kB 26.1MB/s 
    100% |████████████████████████████████| 757kB 18.4MB/s 
    100% |████████████████████████████████| 3.1MB 10.1MB/s 
    100% |████████████████████████████████| 81kB 25.7MB/s 
    100% |████████████████████████████████| 204k

#### Writing A Story


In [2]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy

## good purchase path               <!-- name of the story - for making purchase-->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* purchase                  <! -- user intention to purchase things --->
  - utter_purchase  
* purchase_affirm           <!-- user utterance, in format intent[entities] -->
  - utter_purchase
* purchase_affirm
  - utter_price
* mood_affirm
  - utter_goodbye
  
## good purchase path2
* greet              
  - utter_greet
* inform                    <!-- user utterance, in format intent[entities] -->
  - utter_ask_item
  - utter_show_item
  - utter_did_that_help
* goodbye
    - utter_goodbye

## no purchase path               <!-- name of the story - for making purchase-->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* purchase                  <! -- user intention to purchase things --->
  - utter_purchase  
* purchase_deny           <!-- user utterance, in format intent[entities] -->
  - utter_purchase
* purchase_deny
  - utter_ask_item
* mood_affirm
  - utter_goodbye
  
  
## get weather of location
* greet
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* inform
  - utter_ask_location
* inform{"location": "Paris"}
  - utter_ask_location
* goodbye
    - utter_goodbye
    - export
    
## get weather of location
* greet
    - utter_greet
* inform[location=London]
    - utter_ask_location
* goodbye
    - utter_goodbye
    - export
  
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

"""

%store stories_md > my_stories.md

Writing 'stories_md' (str) to file 'my_stories.md'.


### Defining the Domain -------------->>>>>>
 - - intents : what you expect users to say,their intentions
 - - action : what your bot is supposed to do(list them eg utter_greet,custom actions)
 - - template :

+ Stories

In [3]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy
  - purchase
  - purchase_affirm
  - purchase_deny
  - inform
  - weather
  
entities:
 - location
 - money
 
slots:
 location:
   type: text


actions:
- utter_greet
- utter_cheer_up
- utter_purchase
- utter_price
- utter_did_that_help
- utter_ask_item
- utter_show_item
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_location

templates:
  utter_greet:
  - text: "Hey! How are you?"
  - text: "Hi How are you?"
  - text: "What's up"
  - text: "Bonjour"

  utter_cheer_up:
  - text: "Stay motivated,all is well"
  - text: "Here is something to make you happy"
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  - text: "How do you mean? Can you clarify "
  - text: "Not quite sure what you want. "
  
  utter_happy:
  - text: "Great carry on!" 
  
  utter_purchase:
  - text: 'What item do you want to buy?'
  - text: 'Here are some recommended products'
  - text: 'You want to buy this?'
  - text: 'We only have this?'
  - text: 'Can you give me the specs ?'
  
  utter_ask_item:
  - text: 'What item do you want to buy?'
  - text: 'How many do you want to buy?'
  - text: 'Which type?'

  utter_show_item:
  - text: "Here are some suggestions."
  - text: "Check this one out."
  - text: "We only have this product"
  - text: "Sending sample picture"
    
  utter_price:
  - text: "It will cost you this much"
  - text: "The price is 100"
  - text: "You will pay"
  - text: "Here is the bill"
  - text: "The total cost is 2000, you can pay with your credit card."

  utter_ask_location:
  - text:'In what location?'
  - text:'In which place?'
  - text:'Where precisely?'

  utter_goodbye:
  - text: "Bye"
  - text: "Talk to you later"
  - text: "Cu"
  - text: "Aurevoir"
  - text: "A bientot "
  - text: "goodbye"
  - text: "have a nice day"
  - text: "see you around"
  - text: "bye bye"
  - text: "See you later"
  
"""

%store domain_yml > my_domain.yml

Writing 'domain_yml' (str) to file 'my_domain.yml'.


#### Training your Dialogue Model
+ rasa_core
+ Using Keras
+ Install keras + tensorflow backend

In [4]:
# Load the packages without using fallbackpolicy
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

INFO:apscheduler.scheduler:Scheduler started


In [5]:
# Set the Agent
agent = Agent('my_domain.yml', policies=[KerasPolicy()])

Using TensorFlow backend.


In [6]:
# loading our neatly defined training dialogues
training_data = agent.load_data('my_stories.md')
agent.train(
        training_data,
        validation_split=0.0,
        epochs=400
)

Processed Story Blocks: 100%|██████████| 9/9 [00:00<00:00, 101.83it/s, # trackers=14]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 326/326 [00:19<00:00, 17.02it/s, # actions=283]
INFO:rasa_core.featurizers:Created 283 action examples.
INFO:rasa_core.policies.keras_policy:Fitting model with 283 total samples and a validation split of 0.0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 5, 27)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                7680      
_________________________________________________________________
dense_1 (Dense)              (None, 13)                429       
_________________________________________________________________
activation_1 (Activation)    (None, 13)                0         
Total params: 8,109
Trainable params: 8,109
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
283/283 [==============================] - 2s 6ms/step - loss: 2.4787 - acc: 0.3428
Epoch 2/400
283/283 [==============================] - 0s 452us/step - loss: 2.3401 - acc: 0.4311
Epoch 3/400
283/283 [==============================] - 0s 483us/step - loss: 2

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model


In [7]:
# Saving Our Dialogue Models
agent.persist('models/dialogue')

INFO:rasa_core.agent:Persisted model to '/content/models/dialogue'


#### Starting up the bot
+ rasa_core
+ agent
+ dialogue model

In [0]:
# Load Dialogue model into agent
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue')

# With NLU Trained data
# agent = Agent.load('models/dialogue', interpreter=model_directory)

#### Practising or Talking to the bot

In [9]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
Hi 
Great carry on!
Whats up
Not quite sure what you want. 
paris
Great carry on!
okay
Not quite sure what you want. 
stop


### Natural Language Understanding and Intent Classification

In [0]:
# Loading our Packages
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

#### How our dataset can look like
+ Describes what the messages mean/intention
+ json
+ md
+ yml




In [11]:
# Data Set
nludataset_md = """
## intent:greet
- hey
- hello
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- so sad

## intent:purchase
- I need to buy IPhone
- I want to purchase some items
- What product is that?
- You need this watch.
- How much that it cost?
- He bought this house for $2000
- She is selling it
- I need to buy another
- Can you make a request for the product
- How much?
- This is so expensive
- It was very cheap
- He is buying 
- You ordered for this
- buy
- sell
- pay
- purchase
- what is the price
- Tell me the price
- Show me the product

## intent:purchase_affirm
- yes
- yes I need to buy it
- indeed I want to buy
- of course
- that sounds good
- correct
- I will take it

## intent:purchase_deny
- no I don't want it
- never
- I don't think so
- don't like that
- no way 
- not really
- no


## intent:inform
- I need to buy IPhone
- I want to purchase some items
- What product is that?

## intent:weather
- I wonder what is the weather in Vilnius right now?
- what is the weather?
- Tell me the weather
- Is the weather nice in Barcelona today
- I am going to London today and I wonder what is the weather out there?
- Show me the weather in Dublin, please
- Tell me the weather in Vilnius

"""

%store nludataset_md > nludataset.md

Writing 'nludataset_md' (str) to file 'nludataset.md'.


In [12]:
# Loading Dataset
# Dataset can be in json or md
training_data = load_data('nludataset.md')


INFO:rasa_nlu.training_data.loading:Training data format of nludataset.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 113 (11 distinct intents)
	- Found intents: 'weather', 'goodbye', 'mood_unhappy', 'inform', 'purchase_deny', 'greet', 'purchase', 'mood_affirm', 'mood_deny', 'purchase_affirm', 'mood_great'
	- entity examples: 0 (0 distinct entities)
	- found entities: 



In [13]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config && pip install pygraphviz;
!pip install rasa_nlu[spacy];
!python -m spacy download en;

Selecting previously unselected package libgtk2.0-common.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../0-libgtk2.0-common_2.24.32-1ubuntu1_all.deb ...
Unpacking libgtk2.0-common (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgtk2.0-0:amd64.
Preparing to unpack .../1-libgtk2.0-0_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgtk2.0-0:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgail18:amd64.
Preparing to unpack .../2-libgail18_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgail18:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgail-common:amd64.
Preparing to unpack .../3-libgail-common_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgail-common:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libxdot4.
Preparing to unpack .../4-libxdot4_2.40.1-2_amd64.deb ...
Unpacking libxdot4 (2.40.1-2) ...
Selecting previously unselected package libgvc6-plugins-gtk.
Prep


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



#### Configuration File
+ spacy as backend
+ mitie as backend

In [0]:
# Load the Rasa_nlu Packages
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer



#### Load and Configure our Backend with spaCy

In [15]:

training_data = load_data('nludataset.md')
pipeline = [{"name": "nlp_spacy"}, 
            {"name": "tokenizer_spacy"}, 
            {"name": "intent_featurizer_spacy"},  
            {"name": "intent_classifier_sklearn"}]


INFO:rasa_nlu.training_data.loading:Training data format of nludataset.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 113 (11 distinct intents)
	- Found intents: 'weather', 'goodbye', 'mood_unhappy', 'inform', 'purchase_deny', 'greet', 'purchase', 'mood_affirm', 'mood_deny', 'purchase_affirm', 'mood_great'
	- entity examples: 0 (0 distinct entities)
	- found entities: 



In [16]:
trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
interpreter = trainer.train(training_data)
model_directory = trainer.persist('./projects/default/')

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.model:Starting to train component nlp_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component tokenizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_classifier_sklearn


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.4s finished
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into '/content/projects/default/default/model_20190412-191923'


In [17]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.


 #### Lets Chat

In [18]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
Hi
Bonjour
Price of iphone ?
What item do you want to buy?
Check this one out.
Did that help you?
Nope
Great carry on!
Okay
Aurevoir
stop
